In [1]:
 !pip install pyspark==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=918f8534f8bd73e09f166cc8a6c56a395466ba528db44edc6813ce7f747a558e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!pip install findspark

In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum, avg, min, max, mean, count, to_timestamp, lit
from pyspark.sql import Row

In [4]:

spark = SparkSession.builder.getOrCreate()
file_path = '/kaggle/input/titanic/train.csv'

df = ( spark.read
    .option("header", "true")
    .csv(file_path)
   )


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/18 20:16:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#displaying rows
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [6]:
df.select("Name").show(n=1000, truncate=False)

+----------------------------------------------------------------------------------+
|Name                                                                              |
+----------------------------------------------------------------------------------+
|Braund, Mr. Owen Harris                                                           |
|Cumings, Mrs. John Bradley (Florence Briggs Thayer)                               |
|Heikkinen, Miss. Laina                                                            |
|Futrelle, Mrs. Jacques Heath (Lily May Peel)                                      |
|Allen, Mr. William Henry                                                          |
|Moran, Mr. James                                                                  |
|McCarthy, Mr. Timothy J                                                           |
|Palsson, Master. Gosta Leonard                                                    |
|Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)               

In [7]:
#collecting rows
df.take(3)

[Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin=None, Embarked='S'),
 Row(PassengerId='2', Survived='1', Pclass='1', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38', SibSp='1', Parch='0', Ticket='PC 17599', Fare='71.2833', Cabin='C85', Embarked='C'),
 Row(PassengerId='3', Survived='1', Pclass='3', Name='Heikkinen, Miss. Laina', Sex='female', Age='26', SibSp='0', Parch='0', Ticket='STON/O2. 3101282', Fare='7.925', Cabin=None, Embarked='S')]

In [8]:
# Create a new column 'not_married' with 1 if 'Name' contains 'miss' or 'Miss', and 0 otherwise
df = df.withColumn(
    'not_married',
    when(col('Name').contains('miss') | col('Name').contains('Miss'), 1).otherwise(0)
)
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|not_married|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+-----------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|          0|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|          0|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|          1|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|          0|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|     

In [9]:
# Select a specific row and column value
value = df.filter(col('PassengerId') == 3).select('Name').collect()[0]['Name']
value

'Heikkinen, Miss. Laina'

In [10]:
# Select columns by position
df_selected = df.select(df.columns[0:4])
df_selected.show(5)

+-----------+--------+------+--------------------+
|PassengerId|Survived|Pclass|                Name|
+-----------+--------+------+--------------------+
|          1|       0|     3|Braund, Mr. Owen ...|
|          2|       1|     1|Cumings, Mrs. Joh...|
|          3|       1|     3|Heikkinen, Miss. ...|
|          4|       1|     1|Futrelle, Mrs. Ja...|
|          5|       0|     3|Allen, Mr. Willia...|
+-----------+--------+------+--------------------+
only showing top 5 rows



In [11]:
#getting shape
def get_shape(df):
    num_rows = df.count()
    num_cols = len(df.columns)
    return (num_rows, num_cols)

# Get the shape of the DataFrame
shape = get_shape(df_selected)
shape

(891, 4)

In [12]:
# Concatenate DataFrames: add a new row
new_row = spark.createDataFrame([Row(PassengerId=1000, Survived=1, Pclass=3, Name='JJ')])
df_new = new_row.union(df_selected)
df_new.orderBy("PassengerId", ascending=True).show(5)


+-----------+--------+------+--------------------+
|PassengerId|Survived|Pclass|                Name|
+-----------+--------+------+--------------------+
|          1|       0|     3|Braund, Mr. Owen ...|
|         10|       1|     2|Nasser, Mrs. Nich...|
|        100|       0|     2|   Kantor, Mr. Sinai|
|       1000|       1|     3|                  JJ|
|        101|       0|     3|Petranec, Miss. M...|
+-----------+--------+------+--------------------+
only showing top 5 rows



In [13]:
# Sort values
df = df.orderBy(col('Pclass').asc(), col('Survived').desc())
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|not_married|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+-----------+
|         53|       1|     1|Harper, Mrs. Henr...|female|  49|    1|    0|PC 17572|76.7292|  D33|       C|          0|
|        137|       1|     1|Newsom, Miss. Hel...|female|  19|    0|    2|   11752|26.2833|  D47|       S|          1|
|         56|       1|     1|   Woolner, Mr. Hugh|  male|NULL|    0|    0|   19947|   35.5|  C52|       S|          0|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|  113803|   53.1| C123|       S|          0|
|         62|       1|     1| Icard, Miss. Amelie|female|  38|    0|    0|  113572|     80|  B28|    NULL|          1|
+-----------+--------+------+-------------------

In [14]:
# Drop duplicates
df = df.dropDuplicates(['PassengerId'])

In [15]:
# Drop columns
# Remove columns 'a' and 'b'
df = df.drop('Name', 'Ticket')
df.show(5)

+-----------+--------+------+------+----+-----+-----+-------+-----+--------+-----------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Cabin|Embarked|not_married|
+-----------+--------+------+------+----+-----+-----+-------+-----+--------+-----------+
|          1|       0|     3|  male|  22|    1|    0|   7.25| NULL|       S|          0|
|         10|       1|     2|female|  14|    1|    0|30.0708| NULL|       C|          0|
|        100|       0|     2|  male|  34|    1|    0|     26| NULL|       S|          0|
|        101|       0|     3|female|  28|    0|    0| 7.8958| NULL|       S|          1|
|        102|       0|     3|  male|NULL|    0|    0| 7.8958| NULL|       S|          0|
+-----------+--------+------+------+----+-----+-----+-------+-----+--------+-----------+
only showing top 5 rows



In [16]:
# Rename columns
df = df.withColumnRenamed('Pclass', 'Class')
df.show(5)

+-----------+--------+-----+------+----+-----+-----+-------+-----+--------+-----------+
|PassengerId|Survived|Class|   Sex| Age|SibSp|Parch|   Fare|Cabin|Embarked|not_married|
+-----------+--------+-----+------+----+-----+-----+-------+-----+--------+-----------+
|          1|       0|    3|  male|  22|    1|    0|   7.25| NULL|       S|          0|
|         10|       1|    2|female|  14|    1|    0|30.0708| NULL|       C|          0|
|        100|       0|    2|  male|  34|    1|    0|     26| NULL|       S|          0|
|        101|       0|    3|female|  28|    0|    0| 7.8958| NULL|       S|          1|
|        102|       0|    3|  male|NULL|    0|    0| 7.8958| NULL|       S|          0|
+-----------+--------+-----+------+----+-----+-----+-------+-----+--------+-----------+
only showing top 5 rows



In [17]:
# Group and aggregate
grouped_df = df.groupBy('Sex', 'Class','Survived').agg(count('PassengerId').alias('Passengers_count'))
grouped_df.show(50)

+------+-----+--------+----------------+
|   Sex|Class|Survived|Passengers_count|
+------+-----+--------+----------------+
|female|    2|       0|               6|
|female|    1|       0|               3|
|  male|    3|       0|             300|
|female|    2|       1|              70|
|female|    3|       1|              72|
|  male|    2|       1|              17|
|  male|    1|       0|              77|
|female|    1|       1|              91|
|  male|    1|       1|              45|
|  male|    3|       1|              47|
|female|    3|       0|              72|
|  male|    2|       0|              91|
+------+-----+--------+----------------+



In [18]:
#check nulls
# Initialize an empty dictionary to store the counts
null_counts = {}

# Iterate over each column in the DataFrame
for col_name in df.columns:
    # Count the number of null values in the column
    null_count = df.filter(col(col_name).isNull()).count()
    # Store the count in the dictionary
    null_counts[col_name] = null_count

# Print the null counts for each column
for col_name, count in null_counts.items():
    print(f"Column '{col_name}': {count} null values")

Column 'PassengerId': 0 null values
Column 'Survived': 0 null values
Column 'Class': 0 null values
Column 'Sex': 0 null values
Column 'Age': 177 null values
Column 'SibSp': 0 null values
Column 'Parch': 0 null values
Column 'Fare': 0 null values
Column 'Cabin': 687 null values
Column 'Embarked': 2 null values
Column 'not_married': 0 null values


In [19]:
mean_value = df.select(mean(col('Age'))).collect()[0][0]
print(mean_value)
# Replace null values with the mean value
df = df.withColumn('Age', when(col('Age').isNull(), mean_value).otherwise(col('Age')))

29.69911764705882


In [20]:
# Calculate ranks with partitions
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
df = df.withColumn('Age', col('Age').cast('integer'))
windowSpec = Window.partitionBy("Sex").orderBy(col('Age').asc())
df = df.withColumn('rank', rank().over(windowSpec))
df.show(50)

+-----------+--------+-----+------+---+-----+-----+--------+-------+--------+-----------+----+
|PassengerId|Survived|Class|   Sex|Age|SibSp|Parch|    Fare|  Cabin|Embarked|not_married|rank|
+-----------+--------+-----+------+---+-----+-----+--------+-------+--------+-----------+----+
|        470|       1|    3|female|  0|    2|    1| 19.2583|   NULL|       C|          1|   1|
|        645|       1|    3|female|  0|    2|    1| 19.2583|   NULL|       C|          1|   1|
|        173|       1|    3|female|  1|    1|    1| 11.1333|   NULL|       S|          1|   3|
|        382|       1|    3|female|  1|    0|    2| 15.7417|   NULL|       C|          1|   3|
|        120|       0|    3|female|  2|    4|    2|  31.275|   NULL|       S|          1|   5|
|        206|       0|    3|female|  2|    0|    1| 10.4625|     G6|       S|          1|   5|
|        298|       0|    1|female|  2|    1|    2|  151.55|C22 C26|       S|          1|   5|
|        480|       1|    3|female|  2|    0|    1

In [21]:
# Apply a condition
df = df.withColumn('age_rank', when(col('Age') < 12, 'child').otherwise('not_child'))
df.orderBy(col('Age').desc()).show(10)

+-----------+--------+-----+----+---+-----+-----+-------+-----+--------+-----------+----+---------+
|PassengerId|Survived|Class| Sex|Age|SibSp|Parch|   Fare|Cabin|Embarked|not_married|rank| age_rank|
+-----------+--------+-----+----+---+-----+-----+-------+-----+--------+-----------+----+---------+
|        631|       1|    1|male| 80|    0|    0|     30|  A23|       S|          0| 577|not_child|
|        852|       0|    3|male| 74|    0|    0|  7.775| NULL|       S|          0| 576|not_child|
|        494|       0|    1|male| 71|    0|    0|49.5042| NULL|       C|          0| 574|not_child|
|         97|       0|    1|male| 71|    0|    0|34.6542|   A5|       C|          0| 574|not_child|
|        117|       0|    3|male| 70|    0|    0|   7.75| NULL|       Q|          0| 571|not_child|
|        673|       0|    2|male| 70|    0|    0|   10.5| NULL|       S|          0| 571|not_child|
|        746|       0|    1|male| 70|    1|    1|     71|  B22|       S|          0| 571|not_child|


In [22]:
# Create a pivot table
pivot_df = df.groupBy('Class').pivot('Sex').agg(sum('Survived'))
pivot_df.show(5)

+-----+------+----+
|Class|female|male|
+-----+------+----+
|    3|  72.0|47.0|
|    1|  91.0|45.0|
|    2|  70.0|17.0|
+-----+------+----+

